# Zipline order_value 和 order_target_value

*　order_value:買賣股票至指定價值，指定價值 = 一股單價 * 股數。

*　order_target_value:買/賣使手上股票的當下市場價值達到指定目標。

兩個函數的 parameters 都是：

- asset: 股票
- value: 股票市場價值，正long負short
- limit_price: 限價，預設為None
- stop_price: 指損價，預設為None
- style: Execution style，預設為None，也可以做 limit/stop 的功能。如果 limit_price 或 stop_price 已經填了，就不用也不能填這裡
    
和其他 Zipline 的 order 系列函數運作邏輯類似，都是用下單日的收盤價做基準，將價值先轉換為 amount (並用特別的捨去方式取整數，請見order_percent 篇) ，第二天再用第二天的收盤價 (如果有 stop/limit，則是達到條件的那天)，和下單時算出的 amount (如果期間遇到 split，則是amount / ratio 再捨去) 成交。

## 設定環境

In [22]:
import pandas as pd
import datetime
import tejapi
import time
import os
import warnings
warnings.filterwarnings('ignore')

# tej_key-------------------------------------------
tej_key ='your key '
tejapi.ApiConfig.api_key = tej_key  
os.environ['TEJAPI_BASE'] = "https://api.tej.com.tw" 
os.environ['TEJAPI_KEY'] = tej_key

# date----------------------------------------------
# set date
start='2022-09-16'
end='2022-10-16'
os.environ['mdate'] = '20220916 20221016'     

tz = 'UTC'
start_dt, end_dt = pd.Timestamp(start, tz = tz), pd.Timestamp(end, tz = tz)
# calendar------------------------------------------
calendar_name='TEJ'  

# bundle_name---------------------------------------
bundle_name = 'tquant'

from zipline.utils.calendar_utils import get_calendar
if get_calendar(calendar_name).is_session(start_dt)==False:
    start_dt=get_calendar(calendar_name).next_open(start_dt)
    
if get_calendar(calendar_name).is_session(end_dt)==False:
    end_dt=get_calendar(calendar_name).previous_close(pd.Timestamp(end_dt))
    
from zipline.api import *

from zipline import run_algorithm  
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume

from zipline.utils.run_algo import  (get_transaction_detail,
                                     get_record_vars)

from zipline.sources.TEJ_Api_Data import (get_Treasury_Return,
                                          get_Benchmark_Return,)
import numpy as np
import pandas as pd
from zipline.utils import run_algo
from zipline.data import bundles

coid='2603 IR0001'

    
os.environ['ticker'] = coid       #'1101 1102'   #coid

!zipline ingest -b tquant

Merging daily equity files:


[2023-08-08 05:10:01.693450] INFO: zipline.data.bundles.core: Ingesting tquant.


## Order value購買訊號

### 交易策略

#### def handle_data(context, data):

在模擬的第一個時間點（context.i 為 0），對投資組合中的每個資產下單購買，其價值為 40000 元。

if context.i == 0:
    for asset in context.asset:
        order_value(asset, 40000)

在模擬的第十一個時間點（context.i 為 10），對投資組合中的每個資產進行調整，使其持有價值達到 60000 元。

if context.i == 10:
    for asset in context.asset:
        order_target_value(asset, 60000)

在模擬的第十六個時間點（context.i 為 15），對投資組合中的每個資產進行調整，使其持有價值減少到 30000 元。

if context.i == 15:
    for asset in context.asset:
        order_target_value(asset, 30000)

最後，使用 record 函數記錄每個資產的收盤價。然後遞增 context.i 的值，表示模擬已進入下一個時間點。  

record(close=data.current(context.asset, 'close'))
context.i += 1

In [23]:
def initialize(context):
    context.i = 0
    context.tickers = ['2603']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage.FixedSlippage(spread=0.00))
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    if context.i == 0:
        for asset in context.asset:
            order_value(asset, 40000)
            
    if context.i == 10:
        for asset in context.asset:
            order_target_value(asset, 60000)

    if context.i == 15:
        for asset in context.asset:
            order_target_value(asset, 30000)
            
    record(close=data.current(context.asset, 'close'))
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e5
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')

In [24]:
closing_price = tejapi.get('TWN/APIPRCD',coid=['2603'], opts={'columns':['mdate','coid','close_d']}, mdate={'gte':start_dt,'lte':end_dt }, paginate=True)

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

[2023-08-08 05:10:03.688048]: INFO: handle_simulation_end: Simulated 20 trading days
first open: 2022-09-16 01:01:00+00:00
last close: 2022-10-14 05:30:00+00:00


## 回測資料

In [25]:
positions

,,,amount,cost_basis,last_sale_price
date,asset,symbol,,,
2022-09-19 00:00:00+08:00,Equity(0 [2603]),2603,213,169.240825,169.0
2022-09-20 00:00:00+08:00,Equity(0 [2603]),2603,213,169.240825,172.5
2022-09-21 00:00:00+08:00,Equity(0 [2603]),2603,213,169.240825,166.5
2022-09-22 00:00:00+08:00,Equity(0 [2603]),2603,213,169.240825,156.0
2022-09-23 00:00:00+08:00,Equity(0 [2603]),2603,213,169.240825,158.0
2022-09-26 00:00:00+08:00,Equity(0 [2603]),2603,213,169.240825,145.5
2022-09-27 00:00:00+08:00,Equity(0 [2603]),2603,213,169.240825,153.5
2022-09-28 00:00:00+08:00,Equity(0 [2603]),2603,213,169.240825,144.0
2022-09-29 00:00:00+08:00,Equity(0 [2603]),2603,213,169.240825,144.0


In [26]:
transactions

,,,amount,dt,price,order_id,commission
date,asset,symbol,,,,,
2022-09-19 00:00:00+08:00,Equity(0 [2603]),2603,213,2022-09-19 13:30:00+08:00,169.0,0ce66796a0c743b6946dd3ab9340a704,None
2022-10-03 00:00:00+08:00,Equity(0 [2603]),2603,197,2022-10-03 13:30:00+08:00,149.5,326dd735ef3447ba899a5a0faeec476f,None
2022-10-11 00:00:00+08:00,Equity(0 [2603]),2603,-217,2022-10-11 13:30:00+08:00,156.0,bc3936f4cd64427099b3b1dddc7e8bc9,None


### 每次交易時的現金持有量

In [14]:
performance[['starting_cash','capital_used','ending_cash']].loc[performance['capital_used']!=0]

,starting_cash,capital_used,ending_cash
2022-09-19 13:30:00+08:00,100000.000000,-36048.295725,63951.704275
2022-10-03 13:30:00+08:00,63951.704275,-29493.468388,34458.235887
2022-10-11 13:30:00+08:00,34458.235887,33803.760900,68261.996787


## Order value 與 Order target value 講解

## 1

9/16時下單買40000的長榮(2603)股票，用當天收盤價計算出應買股數為 40000 / 80.8 = 495。<br>
下個交易日9/19遇到減資，比率是2.314356，所以訂購量調整成 495 / 2.314356 = 213，並以9/19收盤價169 * 213股成交。

In [15]:
closing_price[0:3]

,mdate,coid,close_d
None,,,
0,2022-09-16,2603,80.8
1,2022-09-19,2603,169.0
2,2022-09-20,2603,172.5


In [16]:
performance['orders'][0]

[{'id': 'dbcb3ff4db0145ca842fbf6e8e46efbf',
  'dt': Timestamp('2022-09-16 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-09-16 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 495,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [2603]),
  'status': <ORDER_STATUS.OPEN: 0>}]

In [17]:
performance['orders'][1]

[{'id': 'dbcb3ff4db0145ca842fbf6e8e46efbf',
  'dt': Timestamp('2022-09-19 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-09-16 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 213,
  'filled': 213,
  'commission': 51.295725000000004,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [2603]),
  'status': <ORDER_STATUS.FILLED: 1>}]

## 2

在9/30下了order_target_value，目標60000，首先計算手上2603市值 146 * 213股 = 31098，<br>
再計算還需要 (60000-31098) / 146 = 197股 (捨去)，第二日成交。

In [29]:
print(positions.reset_index().iloc[9])
print(closing_price[10:11])

date               2022-09-30 00:00:00+08:00
asset                       Equity(0 [2603])
symbol                                  2603
amount                                   213
cost_basis                        169.240825
last_sale_price                        146.0
Name: 9, dtype: object
          mdate  coid  close_d
None                          
10   2022-09-30  2603    146.0


In [19]:
performance['orders'][11]

[{'id': 'ca7f74f2d7ed4037a4d79689c9df2244',
  'dt': Timestamp('2022-10-03 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-09-30 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 197,
  'filled': 197,
  'commission': 41.968387500000006,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [2603]),
  'status': <ORDER_STATUS.FILLED: 1>}]

## 3

在10/7下了order_target_value，目標30000。因為手上長榮股票市值超過30000，需要賣掉一定數量來達到目標。用10/7收盤價和10/7時手上持有數量，計算出價值155.5 * 410 = 63755，需要賣掉 (63755 - 30000) / 155 = 217股。在下個交易日10/11，以當天收盤價156賣出，手續費是156 * 217股 * 0.001425 (預先設好的commission_cost) = 48.2391

在表格三可以看到，因為賣出股票得到現金，10/11的capital_used是正的，計算方法就是159 * 217 扣掉手續費48.2391 = 33803.7609，當日收盤時手上現金ending_cash = 開盤時現金starting_cash + 當日capital_used。

In [27]:
print(positions.reset_index().iloc[14])
print(closing_price[15:17])

date               2022-10-07 00:00:00+08:00
asset                       Equity(0 [2603])
symbol                                  2603
amount                                   410
cost_basis                        159.857961
last_sale_price                        155.5
Name: 14, dtype: object
          mdate  coid  close_d
None                          
15   2022-10-07  2603    155.5
16   2022-10-11  2603    156.0


In [28]:
performance['orders'][16]

[{'id': 'bc3936f4cd64427099b3b1dddc7e8bc9',
  'dt': Timestamp('2022-10-11 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-10-07 13:30:00+0800', tz='Asia/Taipei'),
  'amount': -217,
  'filled': -217,
  'commission': 48.2391,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [2603]),
  'status': <ORDER_STATUS.FILLED: 1>}]

### 最終的現金持有量

In [38]:
performance[['starting_cash','capital_used','ending_cash']].iloc[16]

starting_cash    34458.235887
capital_used     33803.760900
ending_cash      68261.996787
Name: 2022-10-11 13:30:00+08:00, dtype: float64